In [ ]:
%load_ext tensorboard

from itertools import product
from model import Net, train
from data import trainset
from distributed_setup import run

HP_BATCH_SIZE = [100,1000,5000]
HP_NUM_WORKERS = [0,4,8,16]


model = Net()

for batch_size, num_workers in product(HP_BATCH_SIZE, HP_NUM_WORKERS):
  run(
      model,
      train,
      trainset,
      {'batch_size': batch_size, 'num_workers': num_workers}
  )

In [3]:
import torch
import torchvision
from model import Net, test
from data import testloader, classes
from utils import imshow

PATH = '/project/studios/this_studio/image_classifier/experiments/fabric-batch_size=5000-num_workers=16.pth'


dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

reloaded_model = Net()
reloaded_model.load_state_dict(torch.load(PATH))

outputs = reloaded_model(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))

test(reloaded_model)

RuntimeError: Detected that PyTorch and torchvision were compiled with different CUDA versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=11.7. Please reinstall the torchvision that matches your PyTorch install.